In [ ]:
!pip install tensorflow-addons

In [ ]:
import pandas as pd
import numpy as np
from itertools import product, chain

import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras import backend as K


from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.layers import Dropout, concatenate
from tensorflow.keras.layers import Dense, Input, Flatten, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

#sklearn
from sklearn.decomposition import PCA

#plot
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
def dtype(dict_dataset):
    for k in dict_dataset.keys():
        print("{} {}".format(k.ljust(60), type(dict_dataset[k][0])))

In [ ]:
dataset_path = "/content/pazienti_completa.csv"
dataset = pd.read_csv(dataset_path, sep=';').to_dict('list')
dtype(dataset)
dataset_shape = (len(dataset["Nome Pazienti"]), len(dataset.keys()))
print()
print("Dataset shape: {}".format(dataset_shape))

In [ ]:
features_list = ["Coppia centrale assente",                                     
                 "Coppia centrale aggiuntiva",                                   
                 "Microtubulo centrale assente",                                 
                 "Microtubulo centrale aggiuntivo",                              
                 "Dislocazione coppia centrale %",                               
                 "Assenza central sheat %",                                      
                 "ODA+IDA",                                                      
                 "ODA",                                                          
                 "IDA",                                                          
                 "Assonemi a normale ultrastruttura %"
]
targets       = ["Movimento (%) scarsa escursione su campi tot",
                "Movimento (%) circolari su campi tot",
                "Movimento (%) mobili non patologiche",
                "Movimento (%) immobili / virtualmente immobili"
]

X = np.array([ dataset[feature] for feature in features_list]).T
y = np.array([ np.array(dataset[target])/100 for target in targets]).T
print(X.shape)
print(y.shape)

In [ ]:
#example:
print(X[1])
print(y[1])

# NN

In [ ]:
class ReturnBestEarlyStopping(EarlyStopping):
    def __init__(self, **kwargs):
        super(ReturnBestEarlyStopping, self).__init__(**kwargs)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            if self.verbose > 0:
                print(f'\nEpoch {self.stopped_epoch + 1}: early stopping')
        elif self.restore_best_weights:
            if self.verbose > 0:
                print('Restoring model weights from the end of the best epoch.')
            self.model.set_weights(self.best_weights)

In [ ]:
def create_model(hidden_first, hidden_second, regularizer, learning_rate, momentum, nesterov):
  # create model
  model = Sequential()
  model.add(Dense(hidden_first, input_dim=10, activation='sigmoid', kernel_regularizer=l2(regularizer)))
  model.add(Dense(hidden_second, activation='sigmoid', kernel_regularizer=l2(regularizer)))
  model.add(Dense(4, activation='sigmoid'))
	# Compile model
  optimizer = SGD(learning_rate=learning_rate, momentum=momentum, nesterov=nesterov)
  metrics=['accuracy', tfa.metrics.RSquare(multioutput='uniform_average',dtype=tf.float32, y_shape=(4,))]
  model.compile(loss='MeanSquaredError', optimizer=optimizer)
  return model

In [ ]:
best_callback = ReturnBestEarlyStopping(monitor="val_loss", min_delta=0, patience=200, verbose=0, mode="min", restore_best_weights=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=128)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=128)
model = create_model(64, 32, 0.0001, 0.001, 0.8, True)

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=2000, validation_data=(X_val, y_val), callbacks=[best_callback], verbose = 0)

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load dataset
# create model
model = KerasRegressor(build_fn=create_model, verbose=0)
# define the grid search parameters
param_grid = {"hidden_first":  [ 10], 
              "hidden_second": [10], 
              "regularizer":   [0.1], 
              "learning_rate": [0.1], 
              "momentum":      [0.1], 
              "nesterov":      [False, True],
              "epochs":        [3],
              "batch_size":    [32]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, verbose=1)
grid_result = grid.fit(X_train, y_train, validation_split=0.1, callbacks=[best_callback])
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
grid_result.cv_results_

In [ ]:
print(history.history.keys())
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(X_val, y_val)
model.evaluate(X_test, y_test)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from sklearn import mixture


# fit a Gaussian Mixture Model with two components
clf = mixture.GaussianMixture(n_components=3, covariance_type='full')
clf.fit(X_reduced[:,:2])

# display predicted scores by the model as a contour plot
x = np.linspace(-60., 70.)
y = np.linspace(-60., 70.)
X, Y = np.meshgrid(x, y)
XX = np.array([X.ravel(), Y.ravel()]).T
Z = -clf.score_samples(XX)
Z = Z.reshape(X.shape)

CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0), levels=np.logspace(0, 3, 10))
CB = plt.colorbar(CS, shrink=0.8, extend='both')
plt.scatter(X_reduced[:,0], X_reduced[:,1], 12, marker='o')

plt.title('Negative log-likelihood predicted by a GMM')
plt.axis('tight')
plt.show()

In [ ]:
X_reduced[:,:2].shape

test2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=128)
kf = KFold(n_splits=5, random_state=128, shuffle=True)
kfold_index = kf.split(X_train)

In [ ]:
def product_dict(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in product(*vals):
        yield dict(zip(keys, instance))

In [ ]:
def nn_model(hidden_first, hidden_second, regularizer, learning_rate, momentum, nesterov):
  # create model
  model = Sequential()
  model.add(Dense(hidden_first, input_dim=10, activation='sigmoid', kernel_regularizer=l2(regularizer)))
  model.add(Dense(hidden_second, activation='sigmoid', kernel_regularizer=l2(regularizer)))
  model.add(Dense(4, activation='sigmoid'))
	# Compile model
  optimizer = SGD(learning_rate=learning_rate, momentum=momentum, nesterov=nesterov)
  metrics=['accuracy', tfa.metrics.RSquare(multioutput='uniform_average',dtype=tf.float32, y_shape=(4,))]
  model.compile(loss='MeanSquaredError', optimizer=optimizer)
  return model


param_grid_dict = {"hidden_first":  [10,  30, 60, 150, 200, 300, 500, 1000], 
                   "hidden_second": [10, 30, 60, 150, 200, 300, 500, 1000], 
                   "regularizer":   [0.1, 0.001, 0.0001, 0.00001], 
                   "learning_rate": [0.1, 0.01, 0.001, 0.0001], 
                   "momentum":      [0.1, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8], 
                   "nesterov":      [False, True],          
}

param_grid = list(product_dict(**param_grid_dict))
param_grid[0]
for hyper_param in param_grid:
  print("Start Grid Combination n.")
  for train_index_fold, val_index_fold in kfold_index:
    X_trainf = X_train[train_index_fold]
    Y_trainf = y_train[train_index_fold]

    X_valf = X_train[val_index_fold]
    Y_valf = y_train[val_index_fold]
    print(hyper_param)
    model = nn_model(**hyper_param)
    history = model.fit(X_trainf, Y_trainf, batch_size=32, epochs=2000, validation_data=(X_valf, Y_valf), callbacks=[best_callback], verbose = 0)
    model.evaluate(X_trainf, Y_trainf)
    model.evaluate(X_valf, Y_valf)
